# Output path

In [1]:
from pathlib import Path
import os
OUT_PATH = Path('../impl/public/datasets')

# Helpers

In [33]:
import random as rnd
import json
import math
import os

def delete_files(path):
    for root, directories, files in os.walk(path):
        for file in files:
            os.remove(os.path.join(root, file))

def generate_random_paintings(size, maxWidth, maxHeight, maxRatio):
    res = []
    for i in range(size):
        while True:
            w,h = rnd.randint(1, maxWidth), rnd.randint(1, maxHeight)
            if max(w,h) / min(w,h) <= maxRatio:
                break
        res.append({'ident': str(i+1), 'width': w, 'height': h})
    return res

def generate_random_flow(src, dst, flowMin, flowMax):
    return {'from': src, 'to': dst, 'flow': float(f'{rnd.uniform(flowMin, flowMax):.3f}')}

def generate_random_flows(paintings, flowMin, flowMax):
    res = []
    for i in range(len(paintings)):
        for j in range(i+1, len(paintings)):
            res.append(generate_random_flow(src=paintings[i]['ident'], dst=paintings[j]['ident'], flowMin=flowMin, flowMax=flowMax))
    return res


def generate_layout_dimensions(paintings, layoutAreaRatio, ratio=1.2):
    area_sum = sum([p['width']*p['height'] for p in paintings])
    val = math.sqrt((layoutAreaRatio * area_sum) / ratio)
    return math.ceil(ratio * val), math.floor(val)
    

def generate_random_dataset(size, evalFunction, layoutAreaRatio, maxPaintingWidth, maxPaintingHeight, maxPaintingRatio, flowMin, flowMax):
    paintings = generate_random_paintings(size=size, maxWidth=maxPaintingWidth, maxHeight=maxPaintingHeight, maxRatio=maxPaintingRatio)
    flows = generate_random_flows(paintings, flowMin=flowMin, flowMax=flowMax)
    width, height = generate_layout_dimensions(paintings, layoutAreaRatio)
    return {
       'name': f'random_{size}',
       'source':'https://projects.fit.cvut.cz/theses/4473',
       'data':{
          'layout':{
             'width': width,
             'height': height,
             'evalFunc': evalFunction
          },
          'paintings': paintings,
          'paintingsFlow': flows
       }
    }
    
def print_random_dataset(size, evalFunction, layoutAreaRatio, maxPaintingWidth, maxPaintingHeight, maxPaintingRatio, flowMin, flowMax):
    r = generate_random_dataset(size, evalFunction, layoutAreaRatio, maxPaintingWidth, maxPaintingHeight, maxPaintingRatio, flowMin, flowMax)
    print(json.dumps(r, indent=4))

# 1. Random

In [ ]:
RANDOM_OUT_PATH = OUT_PATH / 'random' 
delete_files(RANDOM_OUT_PATH)

for size in range(10, 121, 1):
    data = generate_random_dataset(
        size=size,
        evalFunction='f(x,y) = 0',
        layoutAreaRatio=1.2,
        maxPaintingWidth=10, maxPaintingHeight=10, maxPaintingRatio=3,
        flowMin=0,flowMax=4
    )
    with open(RANDOM_OUT_PATH / f'{data["name"]}.json', 'w') as f:
        # print(f'Writing to {f.name}')
        json.dump(data, f, indent=4)

# 2. Cluster

In [30]:
def generate_random_cluster_dataset(clusterCount, clusterSize, evalFunction, layoutAreaRatio, maxPaintingWidth, maxPaintingHeight, maxPaintingRatio, flow):
    cluster =  generate_random_dataset(clusterCount*clusterSize, evalFunction, layoutAreaRatio, maxPaintingWidth, maxPaintingHeight, maxPaintingRatio, flow, flow)
    cluster['name'] = f'cluster_{clusterCount}_{clusterSize}'
    cluster['data']['paintingsFlow'] = [item for sublist in [generate_random_flows(paintings=cluster['data']['paintings'][i*clusterSize:(i+1)*clusterSize], flowMin=flow, flowMax=flow) for i in range(clusterCount)] for item in sublist]
    return cluster
    

CLUSTER_OUT_PATH = OUT_PATH / 'cluster' 
delete_files(CLUSTER_OUT_PATH)

for clusterCount, clusterSize in [(3,10), (4, 20)]:
    data = generate_random_cluster_dataset(
        clusterCount=clusterCount, clusterSize=clusterSize,
        evalFunction='f(x,y) = 0',
        layoutAreaRatio=1.2,
        maxPaintingWidth=10, maxPaintingHeight=10, maxPaintingRatio=3,
        flow=10)

    with open(CLUSTER_OUT_PATH / f'{data["name"]}.json', 'w') as f:
        # print(f'Writing to {f.name}')
        json.dump(data, f, indent=4)

# 3. Packing

In [37]:
PACKING_OUT_PATH = OUT_PATH / 'packing' 
delete_files(PACKING_OUT_PATH)

for size in range(10, 121, 1):
    data = generate_random_dataset(
        size=size,
        evalFunction='f(x,y) = 0',
        layoutAreaRatio=1,
        maxPaintingWidth=10, maxPaintingHeight=10, maxPaintingRatio=3,
        flowMin=0,flowMax=4
    )
    data['name'] = f'packing_{size}'
    with open(PACKING_OUT_PATH / f'{data["name"]}.json', 'w') as f:
        # print(f'Writing to {f.name}')
        json.dump(data, f, indent=4)

# 4. Biased sparse clustering

In [51]:
BIASED_SPARSE_CLUSTER_OUT_PATH = OUT_PATH / 'biased_sparse_cluster'
delete_files(BIASED_SPARSE_CLUSTER_OUT_PATH)

clusterCount = 3
clusterSize  = 10

data = generate_random_cluster_dataset(
    clusterCount=clusterCount, clusterSize=clusterSize,
    evalFunction='TODO',
    layoutAreaRatio=2,
    maxPaintingWidth=10, maxPaintingHeight=10, maxPaintingRatio=3,
    flow=10)


w,h = data['data']['layout']['width'], data['data']['layout']['height']
val = 100
data['data']['layout']['evalFunc'] = f'f(x,y) = iff(x<{w/3:.2f} & y<{h/2:.2f}, {val}; x>={w/3:.2f} & x<={2*(w/3):.2f} & y>={h/2:.2f}, {val}; x>{2*(w/3):.2f} & y<{h/2:.2f}, {val}; 1, 0)'
data['name'] = f'biased_sparse_{data["name"]}'


with open(BIASED_SPARSE_CLUSTER_OUT_PATH / f'{data["name"]}.json', 'w') as f:
    # print(f'Writing to {f.name}')
    json.dump(data, f, indent=4)